In [ ]:
import findspark
findspark.init('/home/spark/spark-2.1.0-bin-hadoop2.7')
import pyspark
#from pyspark.sql import SparkSession

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [ ]:
 #run this once. restart your kernel for any errors.
# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)
sqlContext = SQLContext(sc)

In [ ]:
socket_stream = ssc.socketTextStream("127.0.0.1", 8899)

In [ ]:
lines = socket_stream.window( 20 )

In [ ]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [ ]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( "#" ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("m") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

In [ ]:
ssc.start()    

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
# Only works for Jupyter Notebooks!
%matplotlib inline 


In [ ]:
count = 0
while count < 100:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize = (10,8))
    #plt.show()
    #sns.plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    plt.show()
    count = count + 1

In [ ]:
sc.stop()